<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/282_EPO_DecisionGenerationUtilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Generation Utilities

In [ ]:
"""Decision Generation Utilities

Generate recommendations (scale, iterate, retire, do_not_start) based on analysis results.
"""

from typing import Dict, List, Any, Optional
from datetime import datetime
from config import ExperimentationPortfolioOrchestratorConfig


def generate_experiment_decision(
    experiment_id: str,
    portfolio_entry: Dict[str, Any],
    definition: Dict[str, Any],
    analysis: Dict[str, Any],
    config: ExperimentationPortfolioOrchestratorConfig
) -> Dict[str, Any]:
    """
    Generate decision recommendation for an experiment based on analysis.

    Decision rules (MVP: rule-based):
    - scale: lift >= scale_threshold_lift (default 20%)
    - iterate: lift >= iterate_threshold_lift but < scale_threshold_lift (default 5-20%)
    - retire: lift < retire_threshold_lift (default < -10%)
    - do_not_start: planned experiment with insufficient data or unclear criteria
    """
    status = portfolio_entry.get("status", "unknown")
    owner = portfolio_entry.get("owner", definition.get("owner", "unknown"))

    # Handle planned experiments differently
    if status == "planned":
        # Check if experiment has clear success criteria
        success_criteria = definition.get("success_criteria", "")
        if not success_criteria or success_criteria.strip() == "":
            return {
                "experiment_id": experiment_id,
                "decision": "do_not_start",
                "rationale": "Insufficient data quality and unclear success criteria.",
                "recommended_action": "Refine experiment design before launch.",
                "owner": owner,
                "decision_date": datetime.now().strftime("%Y-%m-%d")
            }
        else:
            # Planned experiment with clear criteria - no decision yet
            return {
                "experiment_id": experiment_id,
                "decision": "pending",
                "rationale": "Experiment is planned but not yet started.",
                "recommended_action": "Proceed with experiment launch when ready.",
                "owner": owner,
                "decision_date": datetime.now().strftime("%Y-%m-%d")
            }

    # For running/completed experiments, need analysis
    if not analysis:
        return {
            "experiment_id": experiment_id,
            "decision": "pending",
            "rationale": "Experiment analysis not yet available.",
            "recommended_action": "Wait for experiment completion and analysis.",
            "owner": owner,
            "decision_date": datetime.now().strftime("%Y-%m-%d")
        }

    # Get lift percentage
    relative_lift = analysis.get("relative_lift_percent", 0.0)
    direction = analysis.get("direction", "neutral")
    confidence = analysis.get("confidence", "low")

    # Apply decision rules
    if relative_lift >= config.scale_threshold_lift:
        decision = "scale"
        rationale = f"Reply rate increased by {relative_lift:.1f}% with {confidence} confidence. Strong positive results."
        recommended_action = f"Roll out {definition.get('variants', [])[1] if len(definition.get('variants', [])) > 1 else 'treatment'} to all relevant teams."

    elif relative_lift >= config.iterate_threshold_lift:
        decision = "iterate"
        rationale = f"Resolution time improved by {abs(relative_lift):.1f}%, but gains are modest. Further optimization possible."
        recommended_action = "Continue experiment with improved tuning and monitoring."

    elif relative_lift <= config.retire_threshold_lift:
        decision = "retire"
        rationale = f"Experiment showed {abs(relative_lift):.1f}% decline. Not meeting success criteria."
        recommended_action = "Discontinue experiment and consider alternative approaches."

    else:
        # Small negative or neutral result
        decision = "iterate"
        rationale = f"Results are inconclusive ({relative_lift:.1f}% change). More data or refinement needed."
        recommended_action = "Continue monitoring or refine experiment design."

    return {
        "experiment_id": experiment_id,
        "decision": decision,
        "rationale": rationale,
        "recommended_action": recommended_action,
        "owner": owner,
        "decision_date": datetime.now().strftime("%Y-%m-%d")
    }


This is a **great place** to slow down and really internalize what’s happening, because this utility is where your orchestrator **starts acting like a decision-maker**, not just an analyst.

---

## Big Picture First (Zoomed Out)

This utility answers **one very important question**:

> “Given what we learned from the experiment, what should we do next?”

It does **not**:

* run the experiment
* analyze metrics
* change the workflow

It **only**:
👉 **turns analysis into a recommendation**

That’s the key.

---

## What This Function Is (Conceptually)

Think of `generate_experiment_decision` as:

> A **rule-based decision referee**

It looks at:

* what kind of experiment this is
* what the results were
* how strong those results are

And then assigns **one clear label**:

* **scale**
* **iterate**
* **retire**
* **do_not_start**
* **pending**

This is exactly what real product, growth, or experimentation teams do — but now it’s formalized and repeatable.

---

## Step-by-Step, Plain English

### 1️⃣ Inputs = Everything the Decision Needs

```python
experiment_id
portfolio_entry
definition
analysis
config
```

This is very intentional.

Notice:

* No file loading
* No state mutation
* No workflow logic

This function assumes:

> “All the thinking *before* me is already done.”

That’s **clean orchestration**.

---

### 2️⃣ First Big Fork: Planned vs Active Experiments

```python
status = portfolio_entry.get("status")
```

If the experiment is **planned**:

* No results yet
* No lift to evaluate
* Decision is about **readiness**, not performance

So the function asks:

* Do we even know what success looks like?

If **no success criteria** → `do_not_start`
If **clear criteria** → `pending`

👉 This is realistic and very human-like behavior.

---

### 3️⃣ If It’s Running or Completed, We Need Analysis

```python
if not analysis:
    decision = "pending"
```

This is a **guardrail**.

It prevents the agent from:

* guessing
* hallucinating decisions
* pretending confidence without data

As an orchestrator architect, this is HUGE:

> **Never let decisions happen without prerequisites.**

---

### 4️⃣ Extract the Signals That Matter

```python
relative_lift
direction
confidence
```

These are the **only three things** that matter for the MVP decision logic.

This is important:

* You’re **compressing complexity**
* Reducing analysis → signals → action

That’s what good systems do.

---

### 5️⃣ Apply Clear, Explainable Rules

This is the heart of it.

```python
if lift >= scale_threshold:
    scale
elif lift >= iterate_threshold:
    iterate
elif lift <= retire_threshold:
    retire
else:
    iterate
```

Why this is excellent design:

* Rules are **explicit**
* Thresholds live in **config**
* Decisions are **deterministic**
* Rationales are **human-readable**

No black boxes. No magic.

---

### 6️⃣ The Output Is a Decision Object (Not an Action)

The function returns:

```python
{
  "decision": "...",
  "rationale": "...",
  "recommended_action": "...",
  "owner": "...",
  "decision_date": "..."
}
```

This is critical:

👉 The utility does **not execute** the action
👉 It **recommends** the action

Execution belongs to:

* humans
* downstream agents
* future workflows

That separation is what builds trust.

---

## How This Fits the Orchestrator Architecture

### Utilities (this function)

* Stateless
* Deterministic
* Explainable
* Testable

### Nodes (later)

* Decide **when** to call this
* Decide **which experiments** need decisions
* Write results into **state**
* Route the workflow forward

### State

* Stores the decision
* Makes it visible
* Makes it auditable
* Makes it reusable

---

## What You Should Focus On as an Orchestrator Architect

This function teaches you **five core lessons**:

### ✅ 1. Decisions ≠ Actions

Always separate *thinking* from *doing*.

---

### ✅ 2. Guardrails First, Intelligence Second

Check prerequisites before reasoning.

---

### ✅ 3. Rules Before Models (for MVPs)

Rule-based systems are:

* faster
* safer
* more explainable

You can always swap in an LLM later.

---

### ✅ 4. Config Drives Behavior

Thresholds belong in config, not logic.

That’s how you scale without rewriting code.

---

### ✅ 5. Outputs Must Be Human-Readable

Rationale matters as much as correctness.

That’s how you build **organizational trust**.

---

## One-Sentence Mental Model

> This utility is a **decision factory** that turns clean analysis into clear recommendations — nothing more, nothing less.



In [ ]:
def generate_decisions_for_experiments(
    analyzed_experiments: List[Dict[str, Any]],
    portfolio_lookup: Dict[str, Dict[str, Any]],
    definitions_lookup: Dict[str, Dict[str, Any]],
    analysis_lookup: Dict[str, Dict[str, Any]],
    decisions_lookup: Dict[str, Dict[str, Any]],
    calculated_analyses: List[Dict[str, Any]],
    config: ExperimentationPortfolioOrchestratorConfig
) -> List[Dict[str, Any]]:
    """
    Generate decisions for experiments that need them.

    Uses calculated_analyses if available, otherwise uses existing analysis_lookup.
    """
    generated_decisions = []

    # Merge calculated analyses into analysis lookup for decision generation
    merged_analysis = analysis_lookup.copy()
    for calc_analysis in calculated_analyses:
        exp_id = calc_analysis["experiment_id"]
        merged_analysis[exp_id] = calc_analysis

    # Generate decisions for experiments that need them
    for exp_analysis in analyzed_experiments:
        exp_id = exp_analysis["experiment_id"]

        # Skip if decision already exists
        if exp_id in decisions_lookup:
            continue

        # Skip if doesn't need decision
        if not exp_analysis.get("needs_decision"):
            continue

        # Get required data
        portfolio_entry = portfolio_lookup.get(exp_id)
        definition = definitions_lookup.get(exp_id)
        analysis = merged_analysis.get(exp_id)

        if not portfolio_entry or not definition:
            continue

        # Generate decision
        decision = generate_experiment_decision(
            exp_id,
            portfolio_entry,
            definition,
            analysis,
            config
        )

        generated_decisions.append(decision)

    return generated_decisions



This function is where **all the earlier work actually turns into decisions at scale**. Let’s walk through it cleanly and conceptually.

---

## Big Picture First

If the previous function was:

> “How do I decide for **one** experiment?”

Then this function is:

> “How do I apply that decision logic to **an entire portfolio**, safely and efficiently?”

This is **batch orchestration**, not individual reasoning.

---

## What This Function’s Job Is

In one sentence:

> This function finds *which experiments need decisions* and generates those decisions **without duplicating work or overwriting existing decisions**.

It is very careful and very intentional.

---

## How It Thinks (Step by Step)

### 1️⃣ Start With an Empty Decision List

```python
generated_decisions = []
```

This keeps the function **pure**:

* It doesn’t mutate state
* It only returns new decisions it created

That’s clean orchestration design.

---

### 2️⃣ Merge Fresh Analysis With Existing Analysis

```python
merged_analysis = analysis_lookup.copy()
```

Then:

```python
merged_analysis[exp_id] = calc_analysis
```

Why this matters:

* Some analyses may have been **newly calculated**
* Others may already exist on disk
* Decisions should always use the **most up-to-date analysis**

This merge step ensures:

> “Use the best available truth, regardless of where it came from.”

That’s a very mature design move.

---

### 3️⃣ Loop Over Experiments That Were Already Analyzed

```python
for exp_analysis in analyzed_experiments:
```

Key insight:

* You are **not looping over raw portfolio data**
* You are looping over **validated, pre-checked experiments**

This guarantees:

* Required metadata exists
* Missing data was already flagged earlier

That’s orchestration discipline.

---

### 4️⃣ Skip Anything That Should Not Be Touched

There are **three safety checks**:

#### ❌ Skip if a decision already exists

```python
if exp_id in decisions_lookup:
    continue
```

No overwrites. No surprises.

#### ❌ Skip if it doesn’t need a decision

```python
if not exp_analysis.get("needs_decision"):
    continue
```

Only act when necessary.

#### ❌ Skip if required data is missing

```python
if not portfolio_entry or not definition:
    continue
```

No guessing. No half-decisions.

These skips are what make this **safe to run repeatedly**.

---

### 5️⃣ Generate the Decision (The Payoff)

```python
decision = generate_experiment_decision(...)
```

This is the handoff:

* This function **orchestrates**
* The decision utility **reasons**

Perfect separation of concerns.

---

### 6️⃣ Return Only What’s New

```python
return generated_decisions
```

This is subtle but powerful:

* The node can now:

  * add these to state
  * persist them
  * send them to humans
  * route workflows

This function stays **small, predictable, and reusable**.

---

## Why This Is Excellent Orchestrator Design

This function demonstrates **four elite patterns**:

### ✅ 1. Idempotency

You can run it multiple times safely.

---

### ✅ 2. Incremental Intelligence

It only acts on what’s missing.

---

### ✅ 3. Clear Responsibility

* Utilities decide
* This function coordinates

---

### ✅ 4. Portfolio-Scale Thinking

You’ve moved from:

> “Analyze an experiment”

to:

> “Manage an experimentation system”

That’s the jump from analyst → architect.

---

## Mental Model to Keep

> This function is a **decision dispatcher**:
> it finds the right experiments, feeds them the right inputs, and collects the results — without ever thinking for itself.

That’s exactly how orchestrators should behave.

---

## Final Takeaway for a Top-Tier Orchestrator Developer

What you should internalize here:

* **Nodes coordinate**
* **Utilities reason**
* **State remembers**
* **Config controls behavior**
* **Skipping work is as important as doing work**

You’re building systems that scale **because they know when *not* to act**.


